In [71]:
import sys
import os
import numpy as np

# Add the project root to the path
sys.path.append(os.path.abspath(os.path.join('..')))

# Auto-reload modules so we don't need to restart kernel if we change code
%load_ext autoreload
%autoreload 2

from src.azul.game import AzulGame
from src.azul.constants import BLUE, RED, YELLOW, BLACK, WHITE
print("Setup Complete")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setup Complete


In [72]:
# Create a 2-player game
game = AzulGame(num_players=2)

print(f"Number of Factories: {game.num_factories}")
print(f"Current Player: {game.current_player_idx}")

# Check Factories (Should have 4 tiles each)
print("\n--- Initial Factories ---")
# Remember: Col 0 is 'Empty', Cols 1-5 are colors
print(game.factories)

Number of Factories: 5
Current Player: 0

--- Initial Factories ---
[[0 0 1 2 0 1]
 [0 2 1 1 0 0]
 [0 2 1 0 0 1]
 [0 0 2 0 1 1]
 [0 2 1 0 0 1]]


In [73]:
# 1. Identify whose turn it actually is
current_player_idx = game.current_player_idx
current_player = game.players[current_player_idx]

print(f"It is currently Player {current_player_idx}'s turn.")

# 2. Setup the move
# Find a color in Factory 0
factory_idx = 0
# Get indices where count > 0
available_colors_indices = np.where(game.factories[factory_idx] > 0)[0] 

if len(available_colors_indices) > 0:
    chosen_color = available_colors_indices[0] # Pick first available color
    count = game.factories[factory_idx, chosen_color]

    print(f"Action: Player {current_player_idx} picking Color {chosen_color} ({count} tiles) from Factory {factory_idx}")

    # Action: (Factory, Color, Target Row 0)
    action = (factory_idx, chosen_color, 0) 
    
    # 3. Execute
    game.step(action)
    print("Move executed.")
else:
    print("Factory 0 is empty! Rerun the reset cell.")

It is currently Player 0's turn.
Action: Player 0 picking Color 2 (1 tiles) from Factory 0
Move executed.


In [74]:
from src.azul.constants import ID_TO_COLOR

# 1. Determine who just moved
# Since game.step() automatically passes the turn to the next player,
# the player who actually took the action is the previous index.
just_moved_idx = (game.current_player_idx - 1) % game.num_players
p_active = game.players[just_moved_idx]

print(f"--- Verification for Player {just_moved_idx} (Who just moved) ---")

# 2. Check Player's Row 0 (Pattern Line)
row_idx = 0
color_id = p_active.pattern_lines_color[row_idx]
count = p_active.pattern_lines_count[row_idx]
capacity = p_active.get_row_capacity(row_idx)

# Convert ID to string for readability (e.g., 1 -> "B")
color_name = ID_TO_COLOR.get(color_id, "Empty")

print(f"Row {row_idx} Color: {color_name} (ID: {color_id})")
print(f"Row {row_idx} Count: {count} / {capacity}")

# Check overflow (Floor Line)
if p_active.floor_line_count > 0:
    print(f"Floor Line (Penalty): {p_active.floor_line_count} tiles")
    # Show what is actually in the floor line
    floor_contents = [ID_TO_COLOR[c] for c in p_active.floor_line if c != 0]
    print(f"Floor Contents: {floor_contents}")

# 3. Check Factory (Should be completely empty)
print(f"\n--- Factory {factory_idx} Status ---")
print(f"Raw Vector: {game.factories[factory_idx]}")
is_empty = np.sum(game.factories[factory_idx]) == 0
print(f"Is Empty? {is_empty}") # Should be True

# 4. Check Center (Should have the leftovers from that factory)
print(f"\n--- Center Status ---")
print(f"Raw Vector: {game.center}")

# Create a readable list of what's in the center (e.g., ['R:2', 'K:1'])
center_contents = [f"{ID_TO_COLOR[i]}:{count}" for i, count in enumerate(game.center) if count > 0]
print(f"Readable Contents: {center_contents}")

--- Verification for Player 0 (Who just moved) ---
Row 0 Color: Y (ID: 2)
Row 0 Count: 1 / 1

--- Factory 0 Status ---
Raw Vector: [0 0 0 0 0 0]
Is Empty? True

--- Center Status ---
Raw Vector: [0 0 0 2 0 1]
Readable Contents: ['R:2', 'W:1']


In [75]:
from src.azul.constants import FIRST_PLAYER_TOKEN, RED
print("--- TEST: Player 1 moves from Center with Overflow ---")

# 1. Identify current player (Should be Player 1)
p1_idx = game.current_player_idx
p1 = game.players[p1_idx]
print(f"Current Player: {p1_idx}")

# 2. Check Center content before move
# We expect Red (Color 3) to be there based on previous steps
center_reds = game.center[RED]
print(f"Red tiles in Center: {center_reds}")

if center_reds > 0:
    # 3. Execute Move: Pick RED from CENTER, put in Row 0
    # Logic: Row 0 has capacity 1. We pick ~2 tiles. 1 fits, 1 overflows.
    # Plus we get the -1 token.
    print(f"Action: Player {p1_idx} picks RED from CENTER -> Row 0")
    
    action = (-1, RED, 0) # -1 = Center, RED=3, Row=0
    game.step(action)
    
    # 4. Verification
    print(f"\n--- Verification for Player {p1_idx} ---")
    
    # Check Row 0
    print(f"Row 0 Color: {p1.pattern_lines_color[0]} (Should be 3/Red)")
    print(f"Row 0 Count: {p1.pattern_lines_count[0]} / 1")
    
    # Check Floor Line (The important part)
    print(f"\nFloor Line Count: {p1.floor_line_count}")
    print(f"Floor Line Raw: {p1.floor_line}")
    
    # We expect: [RED, FIRST_PLAYER_TOKEN, 0, 0...]
    # RED = 3, TOKEN = 6
    expected_contains_red = RED in p1.floor_line
    expected_contains_token = FIRST_PLAYER_TOKEN in p1.floor_line
    
    print(f"Floor contains Overflow Red? {expected_contains_red}")
    print(f"Floor contains First Player Token? {expected_contains_token}")
    
else:
    print("Test Skipped: No Red tiles in center (Randomness variance). Reset and try again.")

--- TEST: Player 1 moves from Center with Overflow ---
Current Player: 1
Red tiles in Center: 2
Action: Player 1 picks RED from CENTER -> Row 0

--- Verification for Player 1 ---
Row 0 Color: 3 (Should be 3/Red)
Row 0 Count: 1 / 1

Floor Line Count: 2
Floor Line Raw: [6 3 0 0 0 0 0]
Floor contains Overflow Red? True
Floor contains First Player Token? True


In [76]:
from src.azul.constants import BLUE, YELLOW, RED, GRID_SIZE

print("--- SCORING LOGIC TEST ---")

# 1. Reset a player to clear previous state
test_player = game.players[0]
test_player.reset()

# 2. Setup the "Pre-Scoring" State
# Scenario:
# - Wall has a BLUE tile at (0,0)
# - Wall has a RED tile at (1,1)
# - Pattern Line 1 is FULL with YELLOW tiles.
# - Target: Row 1, Col 0 is Yellow.
# - Result: Placing Yellow at (1,0) should connect to Blue (above) and Red (right).

# Manually place existing wall tiles
test_player.wall[0, 0] = BLUE   # Above target
test_player.wall[1, 1] = RED    # Right of target

# Fill Pattern Line 1 (Capacity 2) with Yellow
# Note: In standard wall, Row 1 Col 0 is WHITE, Col 1 is BLUE, Col 2 is YELLOW...
# Wait! Let's check the wall pattern to be precise.
from src.azul.constants import WALL_PATTERN
target_col = np.where(WALL_PATTERN[1] == YELLOW)[0][0] # Find where Yellow goes in Row 1
print(f"Target Column for Yellow in Row 1 is: {target_col}")

# Let's adjust our setup to match the REAL wall pattern
# Row 1 Pattern: White, Blue, Yellow, Red, Black
# So Yellow is at Index 2.
# We need neighbors at (1, 1) [Blue] and (0, 2) [Black].

# New Setup:
# Place Blue at (1, 1) -> Left Neighbor
test_player.wall[1, 1] = BLUE 
# Place Black at (0, 2) -> Top Neighbor
test_player.wall[0, 2] = BLACK

# Now fill Row 1 with Yellow (Target Col 2)
test_player.pattern_lines_color[1] = YELLOW
test_player.pattern_lines_count[1] = 2 # Full

print(f"Pattern Line 1: {test_player.pattern_lines_count[1]} / 2 Yellow Tiles")
print(f"Wall at (1,1): {ID_TO_COLOR[test_player.wall[1,1]]} (Left Neighbor)")
print(f"Wall at (0,2): {ID_TO_COLOR[test_player.wall[0,2]]} (Top Neighbor)")

# 3. Execute Scoring
print("\n... Calculating Bonuses ...")
points = test_player.calculate_round_bonuses()

# 4. Verification
print(f"Points Scored: {points}")

# Expected:
# Horizontal: Yellow + Blue = 2 points
# Vertical: Yellow + Black = 2 points
# Total: 4 points
expected_score = 4

if points == expected_score:
    print("SUCCESS: Scoring logic matches expected adjacency rules!")
else:
    print(f"FAILURE: Expected {expected_score}, got {points}")

# Check final state
print(f"Wall at Target (1, {target_col}): {ID_TO_COLOR[test_player.wall[1, target_col]]}")
print(f"Pattern Line 1 Count: {test_player.pattern_lines_count[1]} (Should be 0)")

--- SCORING LOGIC TEST ---
Target Column for Yellow in Row 1 is: 2
Pattern Line 1: 2 / 2 Yellow Tiles
Wall at (1,1): B (Left Neighbor)
Wall at (0,2): K (Top Neighbor)

... Calculating Bonuses ...
Points Scored: 4
SUCCESS: Scoring logic matches expected adjacency rules!
Wall at Target (1, 2): Y
Pattern Line 1 Count: 0 (Should be 0)
